In [1]:
# on vous donne la solution toute faite !
from pulp import *
from gen_random import gen_random as gen_r

In [2]:
size_graph=10
size_cities=5
max_value=10
prob=0.2


random_graph,cities,graph_complet_cities = gen_r(size_graph,size_cities, max_value, prob)

In [3]:
print(random_graph)
print(cities)
print(graph_complet_cities)

[[inf  3. inf  1. inf inf  3. inf inf inf]
 [ 3. inf  9. inf inf  6. inf  4. inf inf]
 [inf  9. inf inf  6. 10. inf  7. inf  6.]
 [ 1. inf inf inf  1. inf inf inf inf inf]
 [inf inf  6.  1. inf 10. inf inf inf inf]
 [inf  6. 10. inf 10. inf  3.  5. inf inf]
 [ 3. inf inf inf inf  3. inf inf inf inf]
 [inf  4.  7. inf inf  5. inf inf 10.  8.]
 [inf inf inf inf inf inf inf 10. inf  1.]
 [inf inf  6. inf inf inf inf  8.  1. inf]]
[5, 2, 9, 6, 3]
[[inf 10. 13.  3.  7.]
 [10. inf  6. 11.  7.]
 [13.  6. inf 16. 13.]
 [ 3. 11. 16. inf  4.]
 [ 7.  7. 13.  4. inf]]
